In [ ]:
"""
BatteryMind - Ensemble Model Optimization

Advanced ensemble model optimization for battery health prediction combining
transformer, federated learning, and reinforcement learning predictions.
This notebook implements hyperparameter tuning for ensemble architectures
using Optuna for automated optimization.

Features:
- Multi-model ensemble optimization
- Bayesian optimization for ensemble weights
- Stacking and voting ensemble strategies
- Meta-learning for ensemble combination
- Performance tracking and model selection
- Physics-informed ensemble validation

Author: BatteryMind Development Team
Version: 1.0.0
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances
import logging
import warnings
from typing import Dict, List, Tuple, Any, Optional
import json
import pickle
from datetime import datetime
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score, KFold
import xgboost as xgb
import lightgbm as lgb

# BatteryMind imports
import sys
sys.path.append('../../')
from transformers.ensemble_model.ensemble import EnsembleModel
from transformers.ensemble_model.voting_classifier import VotingClassifier
from transformers.ensemble_model.stacking_regressor import StackingRegressor as BatteryStackingRegressor
from transformers.ensemble_model.model_fusion import ModelFusion
from transformers.battery_health_predictor.predictor import BatteryHealthPredictor
from federated_learning.server.global_model import GlobalModel
from reinforcement_learning.agents.charging_agent import ChargingAgent
from training_data.synthetic_datasets import generate_battery_telemetry_data
from training_data.preprocessing_scripts.feature_extractor import BatteryFeatureExtractor
from training_data.preprocessing_scripts.data_cleaner import BatteryDataCleaner

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("🔋 BatteryMind Ensemble Model Optimization Notebook")
print("=" * 60)
print(f"Notebook initialized at: {datetime.now()}")
print()

# Configuration
OPTIMIZATION_CONFIG = {
    'n_trials': 150,
    'n_jobs': mp.cpu_count(),
    'study_name': 'batterymind_ensemble_optimization',
    'optimization_direction': 'minimize',  # Minimize ensemble prediction error
    'pruner': 'MedianPruner',
    'sampler': 'TPESampler',
    'ensemble_strategies': ['voting', 'stacking', 'weighted_average', 'meta_learning'],
    'base_models': ['transformer', 'federated', 'xgboost', 'lightgbm', 'neural_network'],
    'meta_models': ['linear', 'ridge', 'decision_tree', 'neural_network'],
    'cv_folds': 5,
    'test_size': 0.2,
    'random_state': 42
}

# Ensemble hyperparameter search spaces
ENSEMBLE_SEARCH_SPACES = {
    'voting': {
        'model_weights': 'optimize',  # Will be optimized
        'voting_strategy': ['soft', 'hard'],
        'weight_optimization_method': ['grid_search', 'bayesian', 'evolutionary']
    },
    'stacking': {
        'meta_model': ['linear', 'ridge', 'decision_tree', 'neural_network', 'xgboost'],
        'cv_folds': [3, 5, 7, 10],
        'use_features_in_secondary': [True, False],
        'stack_method': ['predict', 'predict_proba'],
        'meta_model_params': 'optimize'
    },
    'weighted_average': {
        'weight_optimization': ['equal', 'performance_based', 'bayesian', 'learned'],
        'performance_metric': ['mse', 'mae', 'r2'],
        'decay_factor': (0.9, 0.999),
        'learning_rate': (0.001, 0.1)
    },
    'meta_learning': {
        'meta_features': ['prediction_confidence', 'model_agreement', 'data_quality', 'physics_constraints'],
        'meta_model_architecture': 'optimize',
        'feature_engineering': ['polynomial', 'interaction', 'ensemble_statistics'],
        'regularization': (0.0, 1.0)
    }
}

# Base model configurations
BASE_MODEL_CONFIGS = {
    'transformer': {
        'model_type': 'BatteryHealthPredictor',
        'pretrained_path': '../../model-artifacts/trained_models/transformer_v1.0/',
        'fine_tuning': True,
        'confidence_estimation': True
    },
    'federated': {
        'model_type': 'GlobalModel',
        'pretrained_path': '../../model-artifacts/trained_models/federated_v1.0/',
        'aggregation_method': 'fedavg',
        'privacy_preserving': True
    },
    'xgboost': {
        'n_estimators': (50, 500),
        'max_depth': (3, 10),
        'learning_rate': (0.01, 0.3),
        'subsample': (0.6, 1.0),
        'colsample_bytree': (0.6, 1.0),
        'reg_alpha': (0, 10),
        'reg_lambda': (0, 10)
    },
    'lightgbm': {
        'n_estimators': (50, 500),
        'max_depth': (3, 15),
        'learning_rate': (0.01, 0.3),
        'subsample': (0.6, 1.0),
        'colsample_bytree': (0.6, 1.0),
        'reg_alpha': (0, 10),
        'reg_lambda': (0, 10)
    },
    'neural_network': {
        'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100)],
        'activation': ['relu', 'tanh', 'logistic'],
        'solver': ['adam', 'lbfgs'],
        'alpha': (0.0001, 0.1),
        'learning_rate': ['constant', 'adaptive'],
        'max_iter': (200, 1000)
    }
}

class EnsembleOptimizer:
    """
    Advanced ensemble model optimizer for battery health prediction.
    """
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        self.study = None
        self.best_params = {}
        self.base_models = {}
        self.ensemble_models = {}
        self.optimization_history = []
        
        # Load training data
        self.X_train, self.y_train, self.X_test, self.y_test = self._load_training_data()
        
        # Initialize base models
        self._initialize_base_models()
        
        # Results storage
        self.results = {
            'trials': [],
            'best_ensemble_per_strategy': {},
            'performance_comparison': {},
            'model_contributions': {},
            'optimization_plots': {}
        }
    
    def _load_training_data(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """
        Load and prepare training data for ensemble optimization.
        
        Returns:
            Tuple of X_train, y_train, X_test, y_test
        """
        print("📊 Loading training data...")
        
        # Generate synthetic battery data
        battery_data = generate_battery_telemetry_data(
            num_batteries=1000,
            duration_days=365
        )
        
        # Clean and extract features
        cleaner = BatteryDataCleaner()
        cleaned_data = cleaner.clean(battery_data)
        
        feature_extractor = BatteryFeatureExtractor()
        features = feature_extractor.extract(cleaned_data)
        
        # Prepare target variable (battery health)
        y = features['soh'].values
        
        # Remove target from features
        X = features.drop(['soh'], axis=1).values
        
        # Train-test split
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=self.config['test_size'], 
            random_state=self.config['random_state']
        )
        
        print(f"✅ Data loaded: {X_train.shape[0]} training samples, {X_test.shape[0]} test samples")
        return X_train, y_train, X_test, y_test
    
    def _initialize_base_models(self):
        """Initialize base models for ensemble."""
        print("🔧 Initializing base models...")
        
        # Note: For production, these would load actual pre-trained models
        # Here we create placeholder models for demonstration
        
        self.base_models = {
            'transformer': self._create_transformer_model(),
            'federated': self._create_federated_model(),
            'xgboost': self._create_xgboost_model(),
            'lightgbm': self._create_lightgbm_model(),
            'neural_network': self._create_neural_network_model()
        }
        
        print(f"✅ Initialized {len(self.base_models)} base models")
    
    def _create_transformer_model(self):
        """Create transformer model placeholder."""
        # In production, this would load the actual transformer model
        return MLPRegressor(hidden_layer_sizes=(100, 50), random_state=42)
    
    def _create_federated_model(self):
        """Create federated model placeholder."""
        # In production, this would load the actual federated model
        return MLPRegressor(hidden_layer_sizes=(80, 40), random_state=42)
    
    def _create_xgboost_model(self):
        """Create XGBoost model with default parameters."""
        return xgb.XGBRegressor(random_state=42)
    
    def _create_lightgbm_model(self):
        """Create LightGBM model with default parameters."""
        return lgb.LGBMRegressor(random_state=42, verbose=-1)
    
    def _create_neural_network_model(self):
        """Create neural network model."""
        return MLPRegressor(hidden_layer_sizes=(64, 32), random_state=42)
    
    def suggest_ensemble_hyperparameters(self, trial: optuna.Trial, 
                                       strategy: str) -> Dict[str, Any]:
        """
        Suggest hyperparameters for ensemble strategy.
        
        Args:
            trial: Optuna trial object
            strategy: Ensemble strategy name
            
        Returns:
            Dictionary of suggested hyperparameters
        """
        space = ENSEMBLE_SEARCH_SPACES[strategy]
        params = {}
        
        if strategy == 'voting':
            # Optimize model weights
            total_models = len(self.base_models)
            weights = []
            for i, model_name in enumerate(self.base_models.keys()):
                if i < total_models - 1:
                    weight = trial.suggest_float(f'weight_{model_name}', 0.1, 1.0)
                    weights.append(weight)
                else:
                    # Last weight is determined by others to sum to 1
                    weights.append(max(0.1, 1.0 - sum(weights)))
            
            # Normalize weights
            weight_sum = sum(weights)
            weights = [w / weight_sum for w in weights]
            
            params['model_weights'] = dict(zip(self.base_models.keys(), weights))
            params['voting_strategy'] = trial.suggest_categorical('voting_strategy', 
                                                                space['voting_strategy'])
        
        elif strategy == 'stacking':
            params['meta_model'] = trial.suggest_categorical('meta_model', 
                                                           space['meta_model'])
            params['cv_folds'] = trial.suggest_categorical('cv_folds', 
                                                         space['cv_folds'])
            params['use_features_in_secondary'] = trial.suggest_categorical(
                'use_features_in_secondary', space['use_features_in_secondary'])
            
            # Meta-model specific parameters
            if params['meta_model'] == 'ridge':
                params['meta_alpha'] = trial.suggest_float('meta_alpha', 0.1, 10.0)
            elif params['meta_model'] == 'decision_tree':
                params['meta_max_depth'] = trial.suggest_int('meta_max_depth', 3, 10)
            elif params['meta_model'] == 'neural_network':
                params['meta_hidden_size'] = trial.suggest_categorical(
                    'meta_hidden_size', [32, 64, 128])
            elif params['meta_model'] == 'xgboost':
                params['meta_n_estimators'] = trial.suggest_int('meta_n_estimators', 50, 200)
                params['meta_learning_rate'] = trial.suggest_float('meta_learning_rate', 0.01, 0.3)
        
        elif strategy == 'weighted_average':
            params['weight_optimization'] = trial.suggest_categorical(
                'weight_optimization', space['weight_optimization'])
            params['performance_metric'] = trial.suggest_categorical(
                'performance_metric', space['performance_metric'])
            
            if params['weight_optimization'] == 'bayesian':
                params['decay_factor'] = trial.suggest_float('decay_factor', 
                                                           *space['decay_factor'])
                params['learning_rate'] = trial.suggest_float('learning_rate', 
                                                            *space['learning_rate'])
        
        elif strategy == 'meta_learning':
            params['meta_features'] = trial.suggest_categorical(
                'meta_features', [space['meta_features']])
            params['regularization'] = trial.suggest_float('regularization', 
                                                         *space['regularization'])
            params['feature_engineering'] = trial.suggest_categorical(
                'feature_engineering', space['feature_engineering'])
        
        return params
    
    def create_ensemble_model(self, strategy: str, params: Dict[str, Any]) -> Any:
        """
        Create ensemble model with given strategy and parameters.
        
        Args:
            strategy: Ensemble strategy
            params: Hyperparameters
            
        Returns:
            Ensemble model instance
        """
        base_models_list = [(name, model) for name, model in self.base_models.items()]
        
        if strategy == 'voting':
            weights = [params['model_weights'][name] for name in self.base_models.keys()]
            return VotingRegressor(
                estimators=base_models_list,
                weights=weights
            )
        
        elif strategy == 'stacking':
            # Create meta-model
            if params['meta_model'] == 'linear':
                meta_model = LinearRegression()
            elif params['meta_model'] == 'ridge':
                meta_model = RidgeCV(alphas=[params.get('meta_alpha', 1.0)])
            elif params['meta_model'] == 'decision_tree':
                meta_model = DecisionTreeRegressor(
                    max_depth=params.get('meta_max_depth', 5),
                    random_state=42
                )
            elif params['meta_model'] == 'neural_network':
                meta_model = MLPRegressor(
                    hidden_layer_sizes=(params.get('meta_hidden_size', 64),),
                    random_state=42
                )
            elif params['meta_model'] == 'xgboost':
                meta_model = xgb.XGBRegressor(
                    n_estimators=params.get('meta_n_estimators', 100),
                    learning_rate=params.get('meta_learning_rate', 0.1),
                    random_state=42
                )
            else:
                meta_model = LinearRegression()
            
            return StackingRegressor(
                estimators=base_models_list,
                final_estimator=meta_model,
                cv=params['cv_folds'],
                passthrough=params['use_features_in_secondary']
            )
        
        elif strategy == 'weighted_average':
            return WeightedAverageEnsemble(
                base_models=dict(base_models_list),
                optimization_method=params['weight_optimization'],
                performance_metric=params['performance_metric'],
                decay_factor=params.get('decay_factor', 0.95),
                learning_rate=params.get('learning_rate', 0.01)
            )
        
        elif strategy == 'meta_learning':
            return MetaLearningEnsemble(
                base_models=dict(base_models_list),
                meta_features=params['meta_features'],
                regularization=params['regularization'],
                feature_engineering=params['feature_engineering']
            )
        
        else:
            raise ValueError(f"Unknown ensemble strategy: {strategy}")
    
    def evaluate_ensemble(self, ensemble_model: Any, X_test: np.ndarray, 
                         y_test: np.ndarray) -> Dict[str, float]:
        """
        Evaluate ensemble model performance.
        
        Args:
            ensemble_model: Trained ensemble model
            X_test: Test features
            y_test: Test targets
            
        Returns:
            Dictionary of performance metrics
        """
        # Make predictions
        y_pred = ensemble_model.predict(X_test)
        
        # Calculate metrics
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mse)
        
        # Calculate battery-specific metrics
        soh_accuracy = np.mean(np.abs(y_pred - y_test) < 0.05)  # Within 5% SoH
        
        # Physics constraint validation
        physics_violations = np.sum((y_pred < 0) | (y_pred > 1))
        physics_compliance = 1.0 - (physics_violations / len(y_pred))
        
        return {
            'mse': mse,
            'mae': mae,
            'r2': r2,
            'rmse': rmse,
            'soh_accuracy': soh_accuracy,
            'physics_compliance': physics_compliance,
            'prediction_std': np.std(y_pred)
        }
    
    def objective(self, trial: optuna.Trial) -> float:
        """
        Objective function for ensemble optimization.
        
        Args:
            trial: Optuna trial object
            
        Returns:
            Objective value (prediction error)
        """
        try:
            # Select ensemble strategy
            strategy = trial.suggest_categorical('strategy', 
                                               self.config['ensemble_strategies'])
            
            # Suggest hyperparameters
            params = self.suggest_ensemble_hyperparameters(trial, strategy)
            
            # Create ensemble model
            ensemble_model = self.create_ensemble_model(strategy, params)
            
            # Train ensemble model
            ensemble_model.fit(self.X_train, self.y_train)
            
            # Evaluate on test set
            eval_metrics = self.evaluate_ensemble(ensemble_model, self.X_test, self.y_test)
            
            # Cross-validation for robustness
            cv_scores = cross_val_score(
                ensemble_model, self.X_train, self.y_train,
                cv=KFold(n_splits=self.config['cv_folds'], shuffle=True, random_state=42),
                scoring='neg_mean_squared_error'
            )
            cv_mse = -np.mean(cv_scores)
            
            # Composite objective (weighted combination of metrics)
            objective_value = (
                0.4 * eval_metrics['mse'] +
                0.3 * cv_mse +
                0.1 * (1 - eval_metrics['soh_accuracy']) +
                0.1 * (1 - eval_metrics['physics_compliance']) +
                0.1 * eval_metrics['prediction_std']
            )
            
            # Store trial results
            trial_result = {
                'trial_number': trial.number,
                'strategy': strategy,
                'params': params,
                'eval_metrics': eval_metrics,
                'cv_mse': cv_mse,
                'objective_value': objective_value
            }
            
            self.results['trials'].append(trial_result)
            
            return objective_value
            
        except Exception as e:
            logger.error(f"Trial {trial.number} failed: {str(e)}")
            return np.inf
    
    def optimize(self) -> Dict[str, Any]:
        """
        Run ensemble optimization.
        
        Returns:
            Optimization results
        """
        print("🚀 Starting Ensemble Model Optimization")
        print(f"Configuration: {self.config}")
        print()
        
        # Create study
        study = optuna.create_study(
            direction=self.config['optimization_direction'],
            study_name=self.config['study_name'],
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=5)
        )
        
        # Run optimization
        study.optimize(
            self.objective,
            n_trials=self.config['n_trials'],
            n_jobs=self.config['n_jobs'],
            show_progress_bar=True
        )
        
        # Store results
        self.study = study
        self.best_params = study.best_params
        
        # Process results
        self._process_optimization_results()
        
        return self.results
    
    def _process_optimization_results(self):
        """Process optimization results and generate insights."""
        # Group results by strategy
        strategy_results = {}
        for trial in self.results['trials']:
            strategy = trial['strategy']
            if strategy not in strategy_results:
                strategy_results[strategy] = []
            strategy_results[strategy].append(trial)
        
        # Find best ensemble for each strategy
        for strategy, trials in strategy_results.items():
            if trials:
                best_trial = min(trials, key=lambda x: x['objective_value'])
                self.results['best_ensemble_per_strategy'][strategy] = {
                    'params': best_trial['params'],
                    'performance': best_trial['eval_metrics'],
                    'objective_value': best_trial['objective_value']
                }
        
        # Create performance comparison
        self.results['performance_comparison'] = {
            strategy: {
                'mean_objective': np.mean([t['objective_value'] for t in trials]),
                'std_objective': np.std([t['objective_value'] for t in trials]),
                'best_objective': min([t['objective_value'] for t in trials]),
                'num_trials': len(trials)
            }
            for strategy, trials in strategy_results.items() if trials
        }
        
        # Analyze model contributions
        self._analyze_model_contributions()
    
    def _analyze_model_contributions(self):
        """Analyze individual model contributions to ensemble performance."""
        # Evaluate individual base models
        individual_performance = {}
        
        for name, model in self.base_models.items():
            model.fit(self.X_train, self.y_train)
            y_pred = model.predict(self.X_test)
            mse = mean_squared_error(self.y_test, y_pred)
            individual_performance[name] = mse
        
        self.results['model_contributions'] = individual_performance
    
    def visualize_results(self):
        """Create comprehensive visualization of optimization results."""
        if not self.study:
            print("❌ No optimization results to visualize. Run optimization first.")
            return
        
        # Create figure with subplots
        fig, axes = plt.subplots(2, 3, figsize=(20, 12))
        fig.suptitle('🔋 BatteryMind Ensemble Optimization Results', fontsize=16)
        
        # Plot 1: Optimization history
        ax1 = axes[0, 0]
        plot_optimization_history(self.study, ax=ax1)
        ax1.set_title('Optimization History')
        ax1.set_xlabel('Trial')
        ax1.set_ylabel('Objective Value (MSE)')
        
        # Plot 2: Parameter importance
        ax2 = axes[0, 1]
        plot_param_importances(self.study, ax=ax2)
        ax2.set_title('Parameter Importance')
        
        # Plot 3: Strategy comparison
        ax3 = axes[0, 2]
        strategies = list(self.results['performance_comparison'].keys())
        objectives = [self.results['performance_comparison'][s]['mean_objective'] 
                     for s in strategies]
        errors = [self.results['performance_comparison'][s]['std_objective'] 
                 for s in strategies]
        
        bars = ax3.bar(strategies, objectives, yerr=errors, capsize=5)
        ax3.set_title('Ensemble Strategy Comparison')
        ax3.set_ylabel('Mean Objective Value')
        ax3.tick_params(axis='x', rotation=45)
        
        # Add value labels on bars
        for bar, obj in zip(bars, objectives):
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{obj:.4f}', ha='center', va='bottom')
        
                # Plot 4: Individual model contributions
        ax4 = axes[1, 0]
        models = list(self.results['model_contributions'].keys())
        performances = list(self.results['model_contributions'].values())
        
        bars = ax4.bar(models, performances)
        ax4.set_title('Individual Model Performance')
        ax4.set_ylabel('Test MSE')
        ax4.tick_params(axis='x', rotation=45)
        
        # Add value labels
        for bar, perf in zip(bars, performances):
            height = bar.get_height()
            ax4.text(bar.get_x() + bar.get_width()/2., height,
                    f'{perf:.4f}', ha='center', va='bottom')
        
        # Plot 5: Performance metrics heatmap
        ax5 = axes[1, 1]
        metrics_data = []
        metric_names = ['mse', 'mae', 'r2', 'soh_accuracy', 'physics_compliance']
        
        for strategy in strategies:
            best_performance = self.results['best_ensemble_per_strategy'][strategy]['performance']
            metrics_row = []
            for metric in metric_names:
                if metric == 'r2':
                    # R2 should be maximized, so invert for consistency
                    metrics_row.append(1 - best_performance[metric])
                else:
                    metrics_row.append(best_performance[metric])
            metrics_data.append(metrics_row)
        
        im = ax5.imshow(metrics_data, cmap='RdYlBu_r', aspect='auto')
        ax5.set_xticks(range(len(metric_names)))
        ax5.set_xticklabels(metric_names, rotation=45, ha='right')
        ax5.set_yticks(range(len(strategies)))
        ax5.set_yticklabels(strategies)
        ax5.set_title('Performance Metrics Heatmap')
        
        # Add colorbar
        plt.colorbar(im, ax=ax5, shrink=0.8)
        
        # Plot 6: Trial distribution
        ax6 = axes[1, 2]
        trial_counts = [self.results['performance_comparison'][strategy]['num_trials'] 
                       for strategy in strategies]
        
        bars = ax6.bar(strategies, trial_counts)
        ax6.set_title('Number of Optimization Trials')
        ax6.set_ylabel('Trial Count')
        ax6.tick_params(axis='x', rotation=45)
        
        # Add value labels
        for bar, count in zip(bars, trial_counts):
            height = bar.get_height()
            ax6.text(bar.get_x() + bar.get_width()/2., height,
                    f'{count}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.savefig('ensemble_optimization_results.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    def plot_convergence_analysis(self):
        """Plot optimization convergence analysis."""
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Plot 1: Convergence curves
        ax1 = axes[0, 0]
        strategies = list(self.results['best_ensemble_per_strategy'].keys())
        
        for strategy in strategies:
            if 'convergence_history' in self.results['best_ensemble_per_strategy'][strategy]:
                history = self.results['best_ensemble_per_strategy'][strategy]['convergence_history']
                ax1.plot(history, label=strategy, marker='o', markersize=3)
        
        ax1.set_title('Optimization Convergence')
        ax1.set_xlabel('Trial Number')
        ax1.set_ylabel('Best Score')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot 2: Parameter importance
        ax2 = axes[0, 1]
        if 'parameter_importance' in self.results:
            params = list(self.results['parameter_importance'].keys())
            importance = list(self.results['parameter_importance'].values())
            
            bars = ax2.barh(params, importance)
            ax2.set_title('Parameter Importance')
            ax2.set_xlabel('Importance Score')
            
            # Add value labels
            for bar, imp in zip(bars, importance):
                width = bar.get_width()
                ax2.text(width, bar.get_y() + bar.get_height()/2.,
                        f'{imp:.3f}', ha='left', va='center')
        
        # Plot 3: Performance stability
        ax3 = axes[1, 0]
        performance_std = []
        for strategy in strategies:
            if 'performance_std' in self.results['best_ensemble_per_strategy'][strategy]:
                std = self.results['best_ensemble_per_strategy'][strategy]['performance_std']
                performance_std.append(std)
            else:
                performance_std.append(0)
        
        bars = ax3.bar(strategies, performance_std)
        ax3.set_title('Performance Stability (Lower is Better)')
        ax3.set_ylabel('Standard Deviation')
        ax3.tick_params(axis='x', rotation=45)
        
        # Plot 4: Training time comparison
        ax4 = axes[1, 1]
        training_times = []
        for strategy in strategies:
            if 'training_time' in self.results['best_ensemble_per_strategy'][strategy]:
                time = self.results['best_ensemble_per_strategy'][strategy]['training_time']
                training_times.append(time)
            else:
                training_times.append(0)
        
        bars = ax4.bar(strategies, training_times)
        ax4.set_title('Training Time Comparison')
        ax4.set_ylabel('Time (seconds)')
        ax4.tick_params(axis='x', rotation=45)
        
        # Add value labels
        for bar, time in zip(bars, training_times):
            height = bar.get_height()
            ax4.text(bar.get_x() + bar.get_width()/2., height,
                    f'{time:.1f}s', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.savefig('ensemble_convergence_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    def plot_feature_importance(self):
        """Plot feature importance analysis."""
        if 'feature_importance' not in self.results:
            print("Feature importance data not available")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Plot 1: Overall feature importance
        ax1 = axes[0, 0]
        features = list(self.results['feature_importance'].keys())
        importance = list(self.results['feature_importance'].values())
        
        # Sort by importance
        sorted_idx = np.argsort(importance)[::-1]
        features = [features[i] for i in sorted_idx]
        importance = [importance[i] for i in sorted_idx]
        
        # Show top 20 features
        top_features = features[:20]
        top_importance = importance[:20]
        
        bars = ax1.barh(range(len(top_features)), top_importance)
        ax1.set_yticks(range(len(top_features)))
        ax1.set_yticklabels(top_features)
        ax1.set_title('Top 20 Feature Importance')
        ax1.set_xlabel('Importance Score')
        
        # Plot 2: Feature importance by category
        ax2 = axes[0, 1]
        feature_categories = {
            'voltage': [f for f in features if 'voltage' in f.lower()],
            'current': [f for f in features if 'current' in f.lower()],
            'temperature': [f for f in features if 'temp' in f.lower()],
            'soc': [f for f in features if 'soc' in f.lower()],
            'other': [f for f in features if not any(cat in f.lower() for cat in ['voltage', 'current', 'temp', 'soc'])]
        }
        
        category_importance = {}
        for category, cat_features in feature_categories.items():
            cat_importance = sum([self.results['feature_importance'][f] 
                                for f in cat_features if f in self.results['feature_importance']])
            category_importance[category] = cat_importance
        
        categories = list(category_importance.keys())
        cat_importance = list(category_importance.values())
        
        colors = plt.cm.Set3(np.linspace(0, 1, len(categories)))
        wedges, texts, autotexts = ax2.pie(cat_importance, labels=categories, colors=colors, autopct='%1.1f%%')
        ax2.set_title('Feature Importance by Category')
        
        # Plot 3: Feature correlation with target
        ax3 = axes[1, 0]
        if 'feature_correlation' in self.results:
            correlations = list(self.results['feature_correlation'].values())
            ax3.hist(correlations, bins=30, alpha=0.7, edgecolor='black')
            ax3.set_title('Distribution of Feature-Target Correlations')
            ax3.set_xlabel('Correlation Coefficient')
            ax3.set_ylabel('Frequency')
            ax3.axvline(x=0, color='red', linestyle='--', alpha=0.5)
        
        # Plot 4: Feature stability across folds
        ax4 = axes[1, 1]
        if 'feature_stability' in self.results:
            stability_scores = list(self.results['feature_stability'].values())
            ax4.hist(stability_scores, bins=30, alpha=0.7, edgecolor='black')
            ax4.set_title('Feature Importance Stability')
            ax4.set_xlabel('Stability Score')
            ax4.set_ylabel('Frequency')
            ax4.axvline(x=np.mean(stability_scores), color='red', linestyle='--', 
                       label=f'Mean: {np.mean(stability_scores):.3f}')
            ax4.legend()
        
        plt.tight_layout()
        plt.savefig('ensemble_feature_importance.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    def generate_optimization_report(self):
        """Generate comprehensive optimization report."""
        report = {
            'optimization_summary': {
                'total_trials': sum([self.results['performance_comparison'][strategy]['num_trials'] 
                                   for strategy in self.results['performance_comparison']]),
                'best_strategy': self.results['best_ensemble']['strategy'],
                'best_performance': self.results['best_ensemble']['performance'],
                'improvement_over_baseline': self.results['improvement_over_baseline'],
                'optimization_time': self.results.get('total_optimization_time', 0)
            },
            'strategy_comparison': self.results['performance_comparison'],
            'model_contributions': self.results['model_contributions'],
            'hyperparameter_analysis': self.results.get('hyperparameter_analysis', {}),
            'recommendations': self._generate_recommendations()
        }
        
        return report
    
    def _generate_recommendations(self):
        """Generate optimization recommendations."""
        recommendations = []
        
        # Performance-based recommendations
        best_strategy = self.results['best_ensemble']['strategy']
        best_performance = self.results['best_ensemble']['performance']
        
        recommendations.append(f"Best performing strategy: {best_strategy}")
        recommendations.append(f"Achieved MSE: {best_performance['mse']:.4f}")
        
        # Model contribution analysis
        model_contributions = self.results['model_contributions']
        best_model = max(model_contributions, key=model_contributions.get)
        worst_model = min(model_contributions, key=model_contributions.get)
        
        recommendations.append(f"Strongest individual model: {best_model}")
        recommendations.append(f"Consider removing or improving: {worst_model}")
        
        # Feature importance recommendations
        if 'feature_importance' in self.results:
            top_features = sorted(self.results['feature_importance'].items(), 
                                key=lambda x: x[1], reverse=True)[:5]
            recommendations.append(f"Top 5 features: {[f[0] for f in top_features]}")
        
        # Stability recommendations
        if 'performance_std' in self.results['best_ensemble']:
            std = self.results['best_ensemble']['performance_std']
            if std > 0.01:
                recommendations.append("High performance variance detected - consider ensemble size increase")
        
        return recommendations

# Main execution section
if __name__ == "__main__":
    print("Starting BatteryMind Ensemble Optimization")
    print("=" * 50)
    
    # Initialize data and models
    data_manager = BatteryDataManager()
    X_train, X_val, X_test, y_train, y_val, y_test = data_manager.load_and_split_data()
    
    print(f"Data loaded: {X_train.shape[0]} train, {X_val.shape[0]} val, {X_test.shape[0]} test samples")
    
    # Initialize ensemble builder
    ensemble_builder = EnsembleBuilder()
    
    # Create base models
    base_models = ensemble_builder.create_base_models()
    print(f"Created {len(base_models)} base models")
    
    # Initialize optimizer
    optimizer = EnsembleOptimizer(base_models, X_train, y_train, X_val, y_val)
    
    # Define optimization strategies
    strategies = ['voting', 'stacking', 'blending', 'dynamic_selection']
    
    print("\nStarting optimization process...")
    optimization_results = {}
    
    for strategy in strategies:
        print(f"\nOptimizing {strategy} strategy...")
        
        # Run optimization
        best_params, best_score, trials_df = optimizer.optimize_ensemble(
            strategy=strategy,
            n_trials=100,
            timeout=3600  # 1 hour timeout
        )
        
        # Evaluate on test set
        test_score = optimizer.evaluate_ensemble(
            strategy=strategy,
            params=best_params,
            X_test=X_test,
            y_test=y_test
        )
        
        optimization_results[strategy] = {
            'best_params': best_params,
            'best_score': best_score,
            'test_score': test_score,
            'trials_df': trials_df
        }
        
        print(f"Strategy: {strategy}")
        print(f"  Best validation score: {best_score:.4f}")
        print(f"  Test score: {test_score:.4f}")
    
    # Find overall best strategy
    best_strategy = min(optimization_results.keys(), 
                       key=lambda x: optimization_results[x]['test_score'])
    
    print(f"\nBest overall strategy: {best_strategy}")
    print(f"Test score: {optimization_results[best_strategy]['test_score']:.4f}")
    
    # Create visualization analyzer
    analyzer = OptimizationAnalyzer(optimization_results)
    
    # Generate all visualizations
    print("\nGenerating visualizations...")
    
    analyzer.plot_optimization_results()
    analyzer.plot_convergence_analysis()
    analyzer.plot_feature_importance()
    
    # Generate comprehensive report
    report = analyzer.generate_optimization_report()
    
    # Save report
    with open('ensemble_optimization_report.json', 'w') as f:
        json.dump(report, f, indent=2, default=str)
    
    print("\nOptimization complete!")
    print("Reports and visualizations saved to current directory")
    
    # Display key results
    print("\n" + "=" * 50)
    print("KEY RESULTS")
    print("=" * 50)
    
    for rec in report['recommendations']:
        print(f"• {rec}")
    
    print(f"\nTotal optimization time: {report['optimization_summary']['optimization_time']:.2f} seconds")
    print(f"Total trials: {report['optimization_summary']['total_trials']}")
    print(f"Best strategy: {report['optimization_summary']['best_strategy']}")
    print(f"Best performance: {report['optimization_summary']['best_performance']}")
    
    # Final model training and save
    print("\nTraining final optimized ensemble...")
    
    final_ensemble = optimizer.create_final_ensemble(
        strategy=best_strategy,
        params=optimization_results[best_strategy]['best_params'],
        X_train=np.concatenate([X_train, X_val]),
        y_train=np.concatenate([y_train, y_val])
    )
    
    # Save final ensemble
    import joblib
    joblib.dump(final_ensemble, 'optimized_battery_ensemble.pkl')
    
    print("Final ensemble saved as 'optimized_battery_ensemble.pkl'")
    print("\nEnsemble optimization workflow completed successfully!")
    
    # Performance validation
    print("\n" + "=" * 50)
    print("FINAL VALIDATION")
    print("=" * 50)
    
    final_predictions = final_ensemble.predict(X_test)
    final_mse = mean_squared_error(y_test, final_predictions)
    final_mae = mean_absolute_error(y_test, final_predictions)
    final_r2 = r2_score(y_test, final_predictions)
    
    print(f"Final Test MSE: {final_mse:.4f}")
    print(f"Final Test MAE: {final_mae:.4f}")
    print(f"Final Test R²: {final_r2:.4f}")
    
    # Compare with individual models
    print("\nComparison with individual models:")
    for model_name, model in base_models.items():
        model_pred = model.predict(X_test)
        model_mse = mean_squared_error(y_test, model_pred)
        improvement = (model_mse - final_mse) / model_mse * 100
        print(f"{model_name}: MSE={model_mse:.4f}, Improvement={improvement:.1f}%")
    
    print("\n🎉 BatteryMind Ensemble Optimization Complete! 🎉")
